In [1]:
from src.data_processing import data_processing, data_dropping
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, Lasso, LinearRegression
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, make_scorer, accuracy_score, mean_squared_error
from nltk.stem import WordNetLemmatizer as Lemmatizer
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor, RandomForestClassifier
import re
from src.logit_gb import LogitGB

In [2]:
with open('data/text.p', 'rb') as file:
    text = pickle.load(file)
with open('data/target.p', 'rb') as file:
    target = pickle.load(file)

In [3]:
with open('data/pred_trimmed.p', 'rb') as file:
    pred = pickle.load(file)

In [4]:
with open('data/predictor.p', 'wb') as file:
    pickle.dump(predictor, file)

NameError: name 'predictor' is not defined

In [4]:
n = 50000
sample_idx = np.random.choice(len(text), n)
train = np.full(n, False)
train_idx = np.random.choice(n, int(.75*n), replace = False)
for idx in train_idx:
    train[idx] = True
test = ~train

In [5]:
pred['has_existing_stories'] = (pred['existing_stories'] > 0).astype(int)

In [6]:
pred['existing_stories'] = pred['existing_stories'].fillna(0)

In [7]:
y_increase = (target['fire_rate_after'] > target['fire_rate_before']).astype(int)
y_decrease = (target['fire_rate_after'] < target['fire_rate_before']).astype(int)

In [8]:
vec = TfidfVectorizer()

In [9]:
num_mat = pred[['existing_stories', 'has_existing_stories']].fillna(0).values
text_mat = vec.fit_transform(text)

In [10]:
y = y_increase.values[sample_idx]
X = text_mat[sample_idx]

In [16]:
m1 = LogisticRegression()

In [17]:
m1.fit(X[train], y[train])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [195]:
m1.predict_proba(X[test])[:,1]

array([0.68855525, 0.71046879, 0.01720043, ..., 0.01162673, 0.01274593,
       0.21061225])

In [18]:
roc_auc_score(y[test], m1.predict_proba(X[test])[:,1]) #logistic: .8386 gb: .8276

0.8414394250469864

In [23]:
X_final = np.concatenate((num_mat[sample_idx], m1.predict_proba(X)[:,1].reshape(-1,1)), axis = 1)

In [20]:
m2 = GradientBoostingClassifier(learning_rate = .13, loss = 'deviance', n_estimators = 500, min_samples_leaf = 4, max_depth = 3)

In [217]:
#m2 = RandomForestClassifier(n_estimators = 50)

In [24]:
m2.fit(X_final[train], y[train])

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.13, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=4, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [25]:
roc_auc_score(y[test], m2.predict_proba(X_final[test])[:,1]) #logistic: .8772 mix: .8749  GB: .8828  .88436

0.8850782190132369

In [26]:
mean_squared_error(y[test], m2.predict_proba(X_final[test])[:,1]) #logistic: .8772 mix: .8749  GB: .8828  .88436

0.08308111800790724

In [89]:
param_grid = {'learning_rate':[.01,.1,.5], 'loss':['deviance', 'exponential'], 'max_depth':[2,3], 'n_estimators':[50,100,150]}

In [27]:
def f1(y_test, y_pred_prob, threshold):
    y_pred = (y_pred_prob > threshold).astype(int)
    return f1_score(y_test, y_pred)

In [29]:
current_best = 0
best_i = 0
for i in range(1000):
    f = f1(y[test], m2.predict_proba(X_final[test])[:,1], i/1000)
    if f > current_best:
        current_best = f
        best_i = i
    

C:\Users\pengj\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [30]:
f1(y[test], m2.predict_proba(X_final[test])[:,1], best_i/1000)

0.6239963294333561

In [11]:
from src.logit_gb import LogitGB

In [12]:
test = LogitGB()

In [13]:
test.fit(X[train], pred.iloc[sample_idx].iloc[train], target.iloc[sample_idx].iloc[train])

(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(37500, 43921)
(37500, 2)
(

C:\Users\pengj\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [25]:
model.predict_proba(pred.iloc[sample_idx].iloc[test], X[test])

(12500, 2)
(12500, 43921)


array([0.29767812, 0.18491825, 0.00788114, ..., 0.02731724, 0.22904753,
       0.03576547])

In [31]:
roc_auc_score(y[test], model.predict_proba(pred.iloc[sample_idx].iloc[test], X[test]))

(12500, 2)
(12500, 43921)


0.88072063842578

# pred.columns